In [1]:
import numpy as np
from tabulate import tabulate
from scipy.special import eval_legendre

In [6]:
data = np.genfromtxt('USpopulation.txt')[1:]

In [7]:
data

array([[1900.   ,   75.995],
       [1910.   ,   91.972],
       [1920.   ,  105.711],
       [1930.   ,  123.203],
       [1940.   ,  131.669],
       [1950.   ,  150.697],
       [1960.   ,  179.323],
       [1970.   ,  203.212],
       [1980.   ,  226.505],
       [1990.   ,  249.633],
       [2000.   ,  281.422],
       [2010.   ,  308.746]])

## Problem 2
### Part (a)

In [8]:
def generate_T(degree, data):
    N = data.shape[0]
    M = degree
    mat = np.zeros((N,M+1))

    for i in range(degree+1):
        mat[:,i] = data**i
    
    return mat

def compute_least_sq(data, targets, degree):
    T = generate_T(degree, data)
    
    betas, resids, _,_ = np.linalg.lstsq(T, targets,rcond=None)
    
    ss_error = np.sum(resids)
    
    y_pred = generate_T(degree, np.array([2020]))@betas
    return y_pred, ss_error


In [9]:
preds, ss_errors = [], []
degrees = np.arange(12)

for degree in degrees:
    y_pred, resid = compute_least_sq(data[:,0], data[:,1], degree)
    preds.append(y_pred[0])
    ss_errors.append(resid)

In [10]:
dat = np.array([np.arange(12).tolist(), ss_errors, preds]).T
headers = ["Degree", "Sum of sq error", "Predicted price"]

# Generate the table in fancy format.
table = tabulate(dat, headers, tablefmt="fancy_grid")

# # Show it.
print(table)

╒══════════╤═══════════════════╤═══════════════════╕
│   Degree │   Sum of sq error │   Predicted price │
╞══════════╪═══════════════════╪═══════════════════╡
│        0 │         64833.1   │     177.341       │
├──────────┼───────────────────┼───────────────────┤
│        1 │          1211.73  │     314.444       │
├──────────┼───────────────────┼───────────────────┤
│        2 │           106.477 │     342.047       │
├──────────┼───────────────────┼───────────────────┤
│        3 │             0     │     342.596       │
├──────────┼───────────────────┼───────────────────┤
│        4 │             0     │     343.076       │
├──────────┼───────────────────┼───────────────────┤
│        5 │             0     │     343.488       │
├──────────┼───────────────────┼───────────────────┤
│        6 │             0     │  -25992.2         │
├──────────┼───────────────────┼───────────────────┤
│        7 │             0     │   19128.8         │
├──────────┼───────────────────┼──────────────

A fourth or fifth degree polynomial seems to provide the best fit.

### Part (b)

In [11]:
conds = []
Ts = []

for degree in degrees:
    T = generate_T(degree, data[:,0])
    Ts.append(T)
    conds.append(np.linalg.cond(T))

dat = np.array([np.arange(12).tolist(), conds]).T
headers = ["Degree", "Condition number"]

# Generate the table in fancy format.
table = tabulate(dat, headers, tablefmt="fancy_grid")

# # Show it.
print(table)

╒══════════╤════════════════════╕
│   Degree │   Condition number │
╞══════════╪════════════════════╡
│        0 │        1           │
├──────────┼────────────────────┤
│        1 │   110752           │
├──────────┼────────────────────┤
│        2 │        1.38617e+10 │
├──────────┼────────────────────┤
│        3 │        1.80046e+15 │
├──────────┼────────────────────┤
│        4 │        2.41813e+20 │
├──────────┼────────────────────┤
│        5 │        3.52671e+25 │
├──────────┼────────────────────┤
│        6 │        1.26134e+30 │
├──────────┼────────────────────┤
│        7 │        2.0031e+36  │
├──────────┼────────────────────┤
│        8 │        1.09297e+36 │
├──────────┼────────────────────┤
│        9 │        1.94202e+39 │
├──────────┼────────────────────┤
│       10 │        4.33137e+43 │
├──────────┼────────────────────┤
│       11 │        1.18455e+47 │
╘══════════╧════════════════════╛


### Part (c)

In [12]:
def generate_T_legendre(degree, data):
    N = data.shape[0]
    M = degree
    mat = np.zeros((N,M+1))

    for i in range(degree+1):
        mat[:,i] = [eval_legendre(i, t) for t in data]
    
    return mat


M = np.array([[1900, 1], [2010, 1]])
a, b = np.linalg.solve(M, np.array([-1,1]))
def tau(t):
    return a*t + b

taus = tau(data[:,0])

In [13]:
conds = []
Ts = []

for degree in degrees:
    T = generate_T_legendre(degree, taus)
    Ts.append(T)
    conds.append(np.linalg.cond(T))
    
dat = np.array([np.arange(12).tolist(), conds]).T
headers = ["Degree", "Condition number"]

# Generate the table in fancy format.
table = tabulate(dat, headers, tablefmt="fancy_grid")

# # Show it.
print(table)

╒══════════╤════════════════════╕
│   Degree │   Condition number │
╞══════════╪════════════════════╡
│        0 │            1       │
├──────────┼────────────────────┤
│        1 │            1.59326 │
├──────────┼────────────────────┤
│        2 │            1.93388 │
├──────────┼────────────────────┤
│        3 │            2.2886  │
├──────────┼────────────────────┤
│        4 │            2.73446 │
├──────────┼────────────────────┤
│        5 │            3.17768 │
├──────────┼────────────────────┤
│        6 │            3.73611 │
├──────────┼────────────────────┤
│        7 │            4.42573 │
├──────────┼────────────────────┤
│        8 │            5.89656 │
├──────────┼────────────────────┤
│        9 │            9.12392 │
├──────────┼────────────────────┤
│       10 │           19.8846  │
├──────────┼────────────────────┤
│       11 │           67.4973  │
╘══════════╧════════════════════╛
